In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data/SpotifyFeatures.csv")

In [3]:
data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [4]:
popularity_class = []

for index, row in data.iterrows():
    if row['popularity'] > 50:
        popularity_class.append(1)
    else:
        popularity_class.append(0)

In [5]:
from collections import Counter
Counter(popularity_class)

Counter({0: 153123, 1: 79602})

In [6]:
data['popularity_class'] = popularity_class

In [7]:
data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence',
       'popularity_class'],
      dtype='object')

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
sc = StandardScaler()
data['duration_ms'] = sc.fit_transform(data[['duration_ms']])
data['loudness'] = sc.fit_transform(data[['loudness']])
data['tempo'] = sc.fit_transform(data[['tempo']])

In [10]:
data = pd.get_dummies(data, columns = ["key", "mode"])

In [11]:
data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,-1.141368,0.910,0.000,...,1,0,0,0,0,0,0,0,1,0
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,-0.821867,0.737,0.000,...,0,0,0,0,0,1,0,0,0,1
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,-0.545298,0.131,0.000,...,0,0,0,0,0,0,0,0,0,1
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,-0.695295,0.326,0.000,...,1,0,0,0,0,0,0,0,1,0
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,-1.282184,0.225,0.123,...,0,0,0,0,1,0,0,0,1,0


In [36]:
data.shape

(232725, 31)

In [39]:
data = data.drop_duplicates(subset=['artist_name', 'track_name'])

In [40]:
data.shape

(176514, 31)

In [41]:
data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'time_signature', 'valence', 'popularity_class', 'key_A', 'key_A#',
       'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E', 'key_F',
       'key_F#', 'key_G', 'key_G#', 'mode_Major', 'mode_Minor'],
      dtype='object')

In [13]:
from sklearn.model_selection import train_test_split

In [42]:
X = data.drop(['genre', 'artist_name', 'track_name', 'track_id', 'time_signature', 'popularity', 'popularity_class'], axis=1)

In [82]:
X_train = X.values[:130000]
y_train = data['popularity_class'][:130000]
X_test = X.values[130000:]
y_test = data['popularity_class'][130000:]

In [83]:
Counter(y_train)

Counter({0: 96572, 1: 33428})

In [86]:
#trying to balance, this is kind of weird rn, it makes the results on the test set worse, but better for top2017
positive_cases = data[:130000][data[:130000]['popularity_class'] == 1]
negative_sample = data[:130000][data[:130000]['popularity_class'] == 0].sample(33428, random_state=0)
train_balanced = pd.concat([positive_cases, negative_sample])

In [87]:
X = train_balanced.drop(['genre', 'artist_name', 'track_name', 'track_id', 'time_signature', 'popularity', 'popularity_class'], axis=1)

In [88]:
X_train = X.values
y_train = train_balanced['popularity_class']

In [71]:
#X_train, X_test, y_train, y_test = train_test_split(X.values, data['popularity_class'], test_size=0.2, random_state=0)

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [89]:
rf = RandomForestClassifier(n_estimators=100)

In [90]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [91]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [92]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [21]:
from sklearn.metrics import accuracy_score

In [93]:
y_pred_rf = rf.predict(X_test)
y_pred_knn = knn.predict(X_test)
y_pred_dtc = dtc.predict(X_test)

print("random forest accuracy", accuracy_score(y_test, y_pred_rf))
print("k nearest neighbors accuracy", accuracy_score(y_test, y_pred_knn))
print("decision tree accuracy", accuracy_score(y_test, y_pred_dtc))

random forest accuracy 0.7112267274369007
k nearest neighbors accuracy 0.5535107709506815
decision tree accuracy 0.640602829255708


In [51]:
top2017 = pd.read_csv("data/featuresdf2017.csv")

In [52]:
popularity_class = [1]*100
top2017['popularity_class'] = popularity_class

top2017['duration_ms'] = sc.fit_transform(top2017[['duration_ms']])
top2017['loudness'] = sc.fit_transform(top2017[['loudness']])
top2017['tempo'] = sc.fit_transform(top2017[['tempo']])

top2017 = pd.get_dummies(top2017, columns = ["key", "mode"])

In [53]:
top2017.head()

,id,name,artists,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_4.0,key_5.0,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,mode_0.0,mode_1.0
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.377358,0.0802,0.5810,0.000000,0.0931,...,0,0,0,0,0,0,0,0,1,0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,0.738776,0.1200,0.2290,0.000000,0.0924,...,0,0,0,0,0,0,0,0,0,1
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,0.499516,0.1700,0.2090,0.000000,0.1120,...,0,0,0,0,0,0,0,0,0,1
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,-0.622604,0.0317,0.0498,0.000014,0.1640,...,0,0,0,0,0,0,0,1,1,0
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,0.763315,0.0367,0.0552,0.000000,0.1670,...,0,0,0,1,0,0,0,0,0,1


In [54]:
X.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'key_A', 'key_A#', 'key_B', 'key_C', 'key_C#', 'key_D',
       'key_D#', 'key_E', 'key_F', 'key_F#', 'key_G', 'key_G#', 'mode_Major',
       'mode_Minor'],
      dtype='object')

In [55]:
X_top2017 = top2017.drop(['id', 'name', 'artists', 'time_signature', 'popularity_class'], axis=1)
X_top2017.columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'key_A', 'key_A#', 'key_B', 'key_C', 'key_C#', 'key_D',
       'key_D#', 'key_E', 'key_F', 'key_F#', 'key_G', 'key_G#', 'mode_Major',
       'mode_Minor']

In [56]:
X_top2017.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor
0,0.825,0.652,1.377358,0.0802,0.5810,0.000000,0.0931,0.931,-0.835063,0.468871,...,0,0,0,0,0,0,0,0,1,0
1,0.694,0.815,0.738776,0.1200,0.2290,0.000000,0.0924,0.813,-1.088400,0.319390,...,0,0,0,0,0,0,0,0,0,1
2,0.660,0.786,0.499516,0.1700,0.2090,0.000000,0.1120,0.846,2.108041,0.300208,...,0,0,0,0,0,0,0,0,0,1
3,0.617,0.635,-0.622604,0.0317,0.0498,0.000014,0.1640,0.446,-0.581871,0.880266,...,0,0,0,0,0,0,0,1,1,0
4,0.609,0.668,0.763315,0.0367,0.0552,0.000000,0.1670,0.811,-1.376289,2.148071,...,0,0,0,1,0,0,0,0,0,1


In [94]:
y_pred_2017_rf = rf.predict(X_top2017)
y_pred_2017_knn = knn.predict(X_top2017)
y_pred_2017_dtc = dtc.predict(X_top2017)

print("random forest accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_rf))
print("k nearest neighbors accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_knn))
print("decision tree accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_dtc))

random forest accuracy (2017 top) 0.37
k nearest neighbors accuracy (2017 top) 0.84
decision tree accuracy (2017 top) 0.28
